<a href="https://colab.research.google.com/github/ozgekokyay/turkish_movie_review_analysis/blob/main/movie_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GRU, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf


In [ ]:
!wget https://www.dropbox.com/s/fz2d3s2ngq8aw2e/turkish_movie_sentiment_dataset.csv?dl=1 -O dataset.csv

In [ ]:
data = pd.read_csv('dataset.csv')
data.head()

In [ ]:
def convert(point):
  p = point.split(',')
  a = int(p[0])*10 + int(p[1])
  if a > 30:
    return 1
  else:
    return 0

In [ ]:
data_point =data['point'].apply(convert)
target = data_point.values.tolist()

comment = data['comment'].values.tolist()


In [ ]:
cutoff = int(len(comment)*0.8)
x_train, x_test = comment[:cutoff], comment[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [ ]:
num_words=10000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(comment)

In [ ]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [ ]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
np.mean(num_tokens), np.max(num_tokens), np.argmax(num_tokens)

In [ ]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

In [ ]:
np.sum(num_tokens<max_tokens)/len(num_tokens)

In [ ]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)
x_train_pad.shape

In [ ]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [ ]:
def tokens_to_string(tokens):
  words =[inverse_map[token] for token in tokens if token!=0]
  text =' '.join(words)
  return text

In [ ]:
from tensorflow.keras.layers import LSTM, Dropout
model = tf.keras.Sequential()
embedding_size = 65
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

model.add(Dense(64, activation='sigmoid'))
model.add(GRU(units=32, return_sequences=True))
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(4, activation='tanh'))

model.add(Dense(1))

model.summary()

In [ ]:
optimizer = Adam(lr=1e-3)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer=optimizer, metrics=['accuracy'])


In [ ]:
from tensorflow.data import Dataset
train_ds = Dataset.zip((Dataset.from_tensor_slices(x_train_pad), Dataset.from_tensor_slices(y_train)))
train_ds = train_ds.shuffle(1024).batch(256)
val_ds = Dataset.zip((Dataset.from_tensor_slices(x_test_pad), Dataset.from_tensor_slices(y_test)))
val_ds = val_ds.batch(256)
steps_per_epoch = len(x_train_pad)//256
validation_steps = len(x_test_pad)//256
model.fit(train_ds.repeat(),  epochs=4, steps_per_epoch=steps_per_epoch, validation_data=val_ds.repeat(), validation_steps=validation_steps)

